# Рыночные признаки

- Бета коэффициент к индексу IMOEX
- Корреляция за 60 дней
- Волатильность индекса

Оценивается связь тикера с общерыночными факторами.


In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Библиотеки загружены")


Библиотеки загружены


## Загрузка данных


In [15]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным
DATA_DIR = current_dir.parent.parent / 'ML' / 'data' / 'processed'
print(f"Путь к данным: {DATA_DIR}")

if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")

# Список всех тикеров
TICKERS = [
    'AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE',
    'GAZP', 'GMKN', 'HYDR', 'IRAO', 'LENT', 'LKOH', 'MAGN',
    'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL',
    'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO',
    'VTBR', 'YNDX'
]
print(f"Всего тикеров: {len(TICKERS)}")


Текущая директория: e:\Python\VolatilityChecker\MOEXScanner\ML\02_feature_engineering
Путь к данным: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed
Директория найдена!
Всего тикеров: 30


## Загрузка и диагностика индекса IMOEX


In [16]:
# Загрузка индекса IMOEX
index_df = pd.read_parquet(DATA_DIR / 'IMOEX_ohlcv_returns.parquet')
print(f"IMOEX загружен: {len(index_df)} записей")
print(f"Период: {index_df['date'].min()} - {index_df['date'].max()}")

# Нормализуем дату
index_df['date_only'] = pd.to_datetime(index_df['date']).dt.normalize()

# Загружаем SBER для сравнения
sber_df = pd.read_parquet(DATA_DIR / 'SBER_ohlcv_returns.parquet')
print(f"\nSBER загружен: {len(sber_df)} записей")
print(f"Период: {sber_df['date'].min()} - {sber_df['date'].max()}")

# Диагностика: сравним количество торговых дней
sber_df['date_only'] = pd.to_datetime(sber_df['date']).dt.normalize()

sber_dates = set(sber_df['date_only'])
imoex_dates = set(index_df['date_only'])

print(f"\n--- ДИАГНОСТИКА ---")
print(f"Уникальных дат SBER: {len(sber_dates)}")
print(f"Уникальных дат IMOEX: {len(imoex_dates)}")
print(f"Общих дат: {len(sber_dates & imoex_dates)}")
print(f"Дат только в SBER: {len(sber_dates - imoex_dates)}")
print(f"Дат только в IMOEX: {len(imoex_dates - sber_dates)}")

# Показать последние даты без совпадений
missing_in_imoex = sorted(sber_dates - imoex_dates)
if missing_in_imoex:
    print(f"\nПоследние 10 дат SBER без данных IMOEX:")
    for d in missing_in_imoex[-10:]:
        print(f"  {d.date()}")


IMOEX загружен: 1253 записей
Период: 2020-10-13 00:00:00 - 2025-10-10 00:00:00

SBER загружен: 1301 записей
Период: 2020-10-13 23:59:59 - 2025-10-11 15:00:27

--- ДИАГНОСТИКА ---
Уникальных дат SBER: 1301
Уникальных дат IMOEX: 1253
Общих дат: 1252
Дат только в SBER: 49
Дат только в IMOEX: 1

Последние 10 дат SBER без данных IMOEX:
  2025-08-31
  2025-09-06
  2025-09-07
  2025-09-13
  2025-09-14
  2025-09-27
  2025-09-28
  2025-10-04
  2025-10-05
  2025-10-11


## Функции расчета рыночных признаков


In [17]:
def calculate_beta(stock_returns, market_returns, window=60, min_periods=None):
    """Расчет бета коэффициента к рынку (индексу)
    
    min_periods: минимальное количество непропущенных значений в окне
    """
    if min_periods is None:
        min_periods = int(window * 0.7)  # Требуем минимум 70% данных в окне
    
    beta_list = []
    
    for i in range(len(stock_returns)):
        if i < window:
            beta_list.append(np.nan)
            continue
        
        stock_window = stock_returns.iloc[i-window:i]
        market_window = market_returns.iloc[i-window:i]
        
        # Убираем NaN для расчёта
        valid_mask = ~(stock_window.isna() | market_window.isna())
        valid_count = valid_mask.sum()
        
        if valid_count < min_periods:
            beta_list.append(np.nan)
            continue
        
        stock_valid = stock_window[valid_mask]
        market_valid = market_window[valid_mask]
        
        covariance = stock_valid.cov(market_valid)
        market_variance = market_valid.var()
        
        beta = covariance / market_variance if market_variance > 0 else np.nan
        beta_list.append(beta)
    
    return pd.Series(beta_list, index=stock_returns.index)


def calculate_correlation(stock_returns, market_returns, window=60, min_periods=None):
    """Скользящая корреляция с индексом (устойчивая к NaN)"""
    if min_periods is None:
        min_periods = int(window * 0.7)
    
    corr_list = []
    
    for i in range(len(stock_returns)):
        if i < window:
            corr_list.append(np.nan)
            continue
        
        stock_window = stock_returns.iloc[i-window:i]
        market_window = market_returns.iloc[i-window:i]
        
        # Убираем NaN
        valid_mask = ~(stock_window.isna() | market_window.isna())
        valid_count = valid_mask.sum()
        
        if valid_count < min_periods:
            corr_list.append(np.nan)
            continue
        
        corr = stock_window[valid_mask].corr(market_window[valid_mask])
        corr_list.append(corr)
    
    return pd.Series(corr_list, index=stock_returns.index)


def market_volatility(market_returns, window=30, min_periods=None):
    """Волатильность рынка (устойчивая к NaN)"""
    if min_periods is None:
        min_periods = int(window * 0.7)
    return market_returns.rolling(window=window, min_periods=min_periods).std() * np.sqrt(252)


def calculate_market_features(df, index_df, windows=[30, 60]):
    """Расчет всех рыночных признаков для тикера"""
    df = df.copy()
    df['date_only'] = pd.to_datetime(df['date']).dt.normalize()
    
    # Объединяем по нормализованной дате
    merged = pd.merge(
        df, 
        index_df[['date_only', 'log_return']], 
        on='date_only', 
        how='left', 
        suffixes=('', '_index')
    )
    merged['index_return'] = merged['log_return_index']
    
    # Расчет признаков для разных окон
    for window in windows:
        merged[f'beta_{window}'] = calculate_beta(
            merged['log_return'], merged['index_return'], window=window
        )
        merged[f'correlation_{window}'] = calculate_correlation(
            merged['log_return'], merged['index_return'], window=window
        )
        merged[f'index_vol_{window}'] = market_volatility(
            merged['index_return'], window=window
        )
    
    # Удаляем вспомогательные колонки
    merged = merged.drop(columns=['date_only', 'log_return_index'], errors='ignore')
    
    return merged

print("Функции рыночных признаков загружены")


Функции рыночных признаков загружены


## Расчет для всех тикеров


In [18]:
results = {}
windows = [30, 60]

for ticker in TICKERS:
    try:
        df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
        df_features = calculate_market_features(df, index_df, windows=windows)
        
        valid_rows = df_features['index_return'].notna().sum()
        results[ticker] = df_features
        print(f"{ticker}: {len(df_features)} записей, {valid_rows} совпадений с индексом")
        
    except Exception as e:
        print(f"{ticker}: ОШИБКА - {e}")

print(f"\nОбработано: {len(results)} из {len(TICKERS)}")


AFKS: 1301 записей, 1252 совпадений с индексом
AFLT: 1301 записей, 1252 совпадений с индексом
ALRS: 1301 записей, 1252 совпадений с индексом
BELU: 1289 записей, 1246 совпадений с индексом
BSPB: 1295 записей, 1250 совпадений с индексом
CHMF: 1301 записей, 1252 совпадений с индексом
FIVE: 242 записей, 193 совпадений с индексом
GAZP: 1301 записей, 1252 совпадений с индексом
GMKN: 1297 записей, 1248 совпадений с индексом
HYDR: 1301 записей, 1252 совпадений с индексом
IRAO: 1299 записей, 1252 совпадений с индексом
LENT: 990 записей, 951 совпадений с индексом
LKOH: 1301 записей, 1252 совпадений с индексом
MAGN: 1301 записей, 1252 совпадений с индексом
MGNT: 1301 записей, 1252 совпадений с индексом
MTSS: 1300 записей, 1252 совпадений с индексом
NLMK: 1301 записей, 1252 совпадений с индексом
NVTK: 1297 записей, 1252 совпадений с индексом
OZON: 1205 записей, 1205 совпадений с индексом
PIKK: 1301 записей, 1252 совпадений с индексом
PLZL: 1297 записей, 1249 совпадений с индексом
ROSN: 1301 записе

## Пример и диагностика


In [19]:
sample_ticker = 'SBER'
if sample_ticker in results:
    df_sample = results[sample_ticker]
    
    print(f"Столбцы: {df_sample.columns.tolist()}")
    
    # Показать данные из середины (где должны быть полные данные)
    print(f"\nПример данных из СЕРЕДИНЫ датасета (где индекс точно есть):")
    mid_idx = len(df_sample) // 2
    print(df_sample[['date', 'close', 'log_return', 'index_return', 'beta_60', 'correlation_60']].iloc[mid_idx:mid_idx+10])
    
    # Показать последние данные
    print(f"\nПоследние 10 записей:")
    print(df_sample[['date', 'close', 'log_return', 'index_return', 'beta_60', 'correlation_60']].tail(10))
    
    # Статистика
    print(f"\n--- СТАТИСТИКА ---")
    print(f"Всего записей: {len(df_sample)}")
    print(f"index_return: {df_sample['index_return'].notna().sum()} значений, {df_sample['index_return'].isna().sum()} NaN")
    
    # Средние значения beta и correlation (без NaN)
    print(f"\nСредние значения (где есть данные):")
    print(f"  beta_60: {df_sample['beta_60'].mean():.3f}")
    print(f"  correlation_60: {df_sample['correlation_60'].mean():.3f}")


Столбцы: ['date', 'open', 'high', 'low', 'close', 'volume', 'value', 'log_return', 'index_return', 'beta_30', 'correlation_30', 'index_vol_30', 'beta_60', 'correlation_60', 'index_vol_60']

Пример данных из СЕРЕДИНЫ датасета (где индекс точно есть):
                   date   close  log_return  index_return   beta_60  \
650 2023-06-05 23:59:59  237.12   -0.028397     -0.009362  1.309478   
651 2023-06-06 23:59:59  240.91    0.015857     -0.004796  1.340594   
652 2023-06-07 23:59:59  241.25    0.001410      0.005000  1.323537   
653 2023-06-08 23:59:59  241.77    0.002153      0.005551  1.320676   
654 2023-06-09 23:59:59  240.40   -0.005683     -0.000609  1.310085   
655 2023-06-13 23:59:59  244.33    0.016216      0.018020  1.348618   
656 2023-06-14 23:59:59  244.60    0.001104      0.002861  1.326412   
657 2023-06-15 23:59:59  245.18    0.002368      0.012574  1.028927   
658 2023-06-16 23:59:59  243.87   -0.005357      0.000221  0.885056   
659 2023-06-19 23:59:59  242.28   -0.006

## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for ticker, df in results.items():
    output_path = OUTPUT_DIR / f"{ticker}_market_features.parquet"
    df.to_parquet(output_path, index=False)
    print(f"Сохранено: {output_path}")

print(f"\nВсе данные сохранены в {OUTPUT_DIR}")
